In [54]:
import pandas as pd
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from scipy.stats import levene
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import  mean_squared_error ,r2_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [37]:
df = pd.read_csv('OnlineNewsPopularity.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head()

,url,ID,Age of Poast,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,global_sentiment_polarity,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,amaM1,731,12,219,0.663594,1.0,0.815385,4,2,1,0,4.680365,0,1,0,0,0,0,5,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,1,0,0,0,0,0,0,0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.045662,0.013699,0.769231,0.230769,0.092562,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,ap M2,731,9,255,0.604743,1.0,0.791946,3,1,1,0,4.913725,0,0,1,0,0,0,4,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1,0,0,0,0,0,0,0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.043137,0.015686,0.733333,0.266667,0.148948,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,appM3,731,9,211,0.575130,1.0,0.663866,3,1,1,0,4.393365,0,0,1,0,0,0,6,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,1,0,0,0,0,0,0,0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.056872,0.009479,0.857143,0.142857,0.323333,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,astM4,731,9,531,0.503788,1.0,0.665635,9,0,1,0,4.404896,0,1,0,0,0,0,7,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1,0,0,0,0,0,0,0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.041431,0.020716,0.666667,0.333333,0.100705,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,attM5,731,13,1072,0.415646,1.0,0.540890,19,19,20,0,4.682836,0,0,0,0,1,0,7,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,1,0,0,0,0,0,0,0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.074627,0.012127,0.860215,0.139785,0.281003,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [38]:
median = df[' shares'].median()
share_label = list()
for share in df[' shares']:
    if share <= median:
        share_label.append(0)
    else:
        share_label.append(1)

# Update this class label into the dataframe
df = pd.concat([df, pd.DataFrame(share_label, columns=['Popularity'])], axis=1)
df.head(3)

,url,ID,Age of Poast,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,global_sentiment_polarity,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,Popularity
0,http://mashable.com/2013/01/07/amazon-instant-...,amaM1,731,12,219,0.663594,1.0,0.815385,4,2,1,0,4.680365,0,1,0,0,0,0,5,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,496.0,496.0,496.0,1,0,0,0,0,0,0,0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.045662,0.013699,0.769231,0.230769,0.092562,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.5,-0.1875,0.0,0.1875,593,0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,ap M2,731,9,255,0.604743,1.0,0.791946,3,1,1,0,4.913725,0,0,1,0,0,0,4,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.043137,0.015686,0.733333,0.266667,0.148948,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.0,0.0000,0.5,0.0000,711,0
2,http://mashable.com/2013/01/07/apple-40-billio...,appM3,731,9,211,0.575130,1.0,0.663866,3,1,1,0,4.393365,0,0,1,0,0,0,6,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,918.0,918.0,918.0,1,0,0,0,0,0,0,0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.056872,0.009479,0.857143,0.142857,0.323333,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.0,0.0000,0.5,0.0000,1500,1


In [39]:
df.corr()

,Age of Poast,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,global_sentiment_polarity,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,Popularity
Age of Poast,1.000000,-0.240320,-0.062867,0.002866,0.000089,0.003805,-0.000832,0.064530,-0.027636,0.000936,0.130465,0.054492,-0.049109,0.055788,0.076287,0.083277,-0.170250,0.046884,0.591199,0.029503,0.133225,-0.076590,-0.637824,-0.493093,-0.157204,-0.051820,-0.163164,-0.011438,-0.014501,-0.015655,-0.006129,-0.005781,0.009961,0.004042,-0.002853,-0.004067,0.004226,0.000272,0.080894,0.004423,-0.141713,-0.030838,0.092906,0.133837,0.207604,0.010266,0.198654,-0.071968,0.158646,0.126344,0.054772,0.098288,-0.000507,0.062175,-0.063239,-0.015919,0.038711,0.011551,-0.002745,0.008662,0.045696
n_tokens_title,-0.240320,1.000000,0.018160,-0.005318,-0.004754,-0.005420,-0.053496,-0.014856,-0.008858,0.051460,-0.071403,-0.070815,0.132791,-0.023902,-0.090394,-0.046716,0.049223,-0.006077,-0.110672,-0.005890,-0.031400,0.012926,0.120841,0.115746,-0.002370,0.006918,0.004296,-0.004563,0.000128,0.000661,0.004274,0.009322,0.008935,-0.015472,-0.002015,-0.015013,0.006289,-0.005996,-0.070038,0.063568,0.038365,0.042208,-0.065063,-0.056804,-0.064951,0.015530,-0.066589,0.034186,-0.072226,-0.049619,-0.025069,-0.021662,-0.017096,-0.029146,0.011425,0.077245,0.000240,-0.146954,0.040550,0.008783,-0.043492
n_tokens_content,-0.062867,0.018160,1.000000,-0.004737,0.017512,0.000373,0.423065,0.304682,0.342600,0.103699,0.167789,0.037548,0.060200,-0.006105,0.033424,0.025408,0.055989,0.072845,-0.054345,0.000066,-0.003545,-0.022786,0.058860,-0.096460,-0.022286,-0.030496,-0.079624,-0.030686,0.025657,-0.013809,-0.002484,-0.004027,-0.016891,-0.007395,-0.015949,0.034538,0.036394,0.052024,0.026218,-0.009724,0.087266,-0.140141,0.041265,0.127879,0.133979,0.125013,0.098960,0.101053,0.021937,0.135123,-0.261493,0.415706,-0.130375,-0.450603,0.225870,0.004484,0.023358,0.007136,0.013439,0.002459,0.040331
n_unique_tokens,0.002866,-0.005318,-0.004737,1.000000,0.999572,0.999852,-0.004352,0.006620,0.018802,-0.000597,0.026407,-0.001653,0.011016,-0.000264,-0.000945,-0.002328,-0.005535,-0.003679,0.001601,-0.000552,-0.000826,0.000577,-0.001624,0.000805,0.004563,-0.002120,-0.002083,0.001036,-0.000222,0.001992,-0.002142,0.010538,-0.002224,-0.002248,-0.001398,-0.002563,-0.001803,-0.003186,-0.002213,-0.000827,-0.006855,-0.003689,-0.004260,-0.000180,0.000014,-0.000877,-0.000667,-0.001657,0.000523,-0.000487,0.009193,-0.009054,0.001453,0.009902,-0.007315,-0.004678,-0.002333,-0.009242,-0.004217,0.000806,0.003226
n_non_stop_words,0.000089,-0.004754,0.017512,0.999572,1.000000,0.999532,0.005521,0.013598,0.028486,-0.000899,0.031554,-0.000314,0.010903,-0.000012,-0.000078,0.000061,-0.002702,-0.001439,-0.000352,-0.000595,-0.000892,-0.000553,0.000394,-0.002939,0.003284,-0.003408,-0.005415,0.000339,0.000170,0.001614,-0.002147,0.010501,-0.002517,-0.002360,-0.001963,-0.001508,-0.001338,-0.002082,-0.001031,-0.002869,-0.003286,-0.009761,-0.001072,0.002565,0.001535,0.001036,0.002421,0.001245,0.000831,0.002285,0.000904,0.002062,-0.001372,-0.001214,-0.000761,-0.005389,-

In [40]:
df.columns=df.columns.str.replace(' ','') 

In [41]:
df.columns

Index(['url', 'ID', 'AgeofPoast', 'n_tokens_title', 'n_tokens_content',
       'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
       'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos',
       'average_token_length', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'num_keywords', 'kw_min_min', 'kw_max_min',
       'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg',
       'kw_max_avg', 'kw_avg_avg', 'self_reference_min_shares',
       'self_reference_max_shares', 'self_reference_avg_sharess',
       'weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday',
       'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday',
       'weekday_is_sunday', 'is_weekend', 'LDA_00', 'LDA_01', 'LDA_02',
       'LDA_03', 'LDA_04', 'global_subjectivity', 'global_rate_positive_words',
       'global_rate_negativ

In [42]:
df1 = df.drop(columns = ['url','AgeofPoast','ID'])

In [43]:
df1.shape

(39644, 60)

In [47]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39644 entries, 0 to 39643
Data columns (total 60 columns):
n_tokens_title                   39644 non-null int64
n_tokens_content                 39644 non-null int64
n_unique_tokens                  39644 non-null float64
n_non_stop_words                 39644 non-null float64
n_non_stop_unique_tokens         39644 non-null float64
num_hrefs                        39644 non-null int64
num_self_hrefs                   39644 non-null int64
num_imgs                         39644 non-null int64
num_videos                       39644 non-null int64
average_token_length             39644 non-null float64
data_channel_is_lifestyle        39644 non-null int64
data_channel_is_entertainment    39644 non-null int64
data_channel_is_bus              39644 non-null int64
data_channel_is_socmed           39644 non-null int64
data_channel_is_tech             39644 non-null int64
data_channel_is_world            39644 non-null int64
num_keywords       

In [44]:
df1.describe()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,global_sentiment_polarity,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,Popularity
count,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
mean,10.398749,546.514731,0.548216,0.996469,0.689175,10.883690,3.293638,4.544143,1.249874,4.548239,0.052946,0.178009,0.157855,0.058597,0.185299,0.212567,7.223767,26.106801,1153.951682,312.366967,13612.354102,752324.066694,259281.938083,1117.146610,5657.211151,3135.858639,3998.755396,10329.212662,6401.697580,0.168020,0.186409,0.187544,0.183306,0.143805,0.061876,0.069039,0.130915,0.184599,0.141256,0.216321,0.223770,0.234029,0.443370,0.039625,0.016612,0.682150,0.287934,0.119309,0.353825,0.095446,0.756728,-0.259524,-0.521944,-0.107500,0.282353,0.071425,0.341843,0.156064,3395.380184,0.493442
std,2.114037,471.107508,3.520708,5.231231,3.264816,11.332017,3.855141,8.309434,4.107855,0.844406,0.223929,0.382525,0.364610,0.234871,0.388545,0.409129,1.909130,69.633215,3857.990877,620.783887,57986.029357,214502.129573,135102.247285,1137.456951,6098.871957,1318.150397,19738.670516,41027.576613,24211.332231,0.373889,0.389441,0.390353,0.386922,0.350896,0.240933,0.253524,0.337312,0.262975,0.219707,0.282145,0.295191,0.289183,0.116685,0.017429,0.010828,0.190206,0.156156,0.096931,0.104542,0.071315,0.247786,0.127726,0.290290,0.095373,0.324247,0.265450,0.188791,0.226294,11626.950749,0.499963
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.393750,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,1.000000,0.000000
25%,9.000000,246.000000,0.470870,1.000000,0.625739,4.000000,1.000000,1.000000,0.000000,4.478404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,-1.000000,445.000000,141.750000,0.000000,843300.000000,172846.875000,0.000000,3562.101631,2382.448565,639.000000,1100.000000,981.187500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025051,0.025012,0.028571,0.028571,0.028574,0.396167

In [48]:
x = df1.drop(columns = 'shares')
y = df1['shares']

In [49]:
import statsmodels.api as sm
xc = sm.add_constant(x)
ols_model = sm.OLS(y,xc).fit()
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 shares   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     43.12
Date:                Wed, 11 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:13:07   Log-Likelihood:            -4.2617e+05
No. Observations:               39644   AIC:                         8.525e+05
Df Residuals:                   39586   BIC:                         8.530e+05
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                         -8.506e+05   5.12e+06     -0.166      0.868   -1.09e+07    9.18e+06
n_tokens_title                   88.2324     28.148      3.135      0.002      33.061     143.403
n_tokens_content                  0.4597      0.219      2.095      0.036       0.030       0.890
n_unique_tokens                3973.8776   1883.545      2.110      0.035     282.084    7665.671
n_non_stop_words              -1123.8373   5803.035     -0.194      0.846   -1.25e+04    1.03e+04
n_non_stop_unique_tokens       -943.4342   1599.824     -0.590      0.555   -4079.128    2192.260
num_hrefs                        18.5773      6.587      2.820      0.005       5.667      31.488
num_self_hrefs                  -34.3386     17.503     -1.962      0.050     -68.645      -0.032
num_imgs                          9.9232      8.779      1.130      0.258      -7.284      27.130
num_videos                        3.8519     15.463      0.249      0.803     -26.456      34.160
average_token_length           -465.2246    238.444     -1.951      0.051    -932.581       2.132
data_channel_is_lifestyle      -866.8313    387.477     -2.237      0.025   -1626.296    -107.367
data_channel_is_entertainment  -779.3576    250.753     -3.108      0.002   -1270.840    -287.875
data_channel_is_bus            -461.3687    375.834     -1.228      0.220   -1198.013     275.276
data_channel_is_socmed        -1321.4819    366.055     -3.610      0.000   -2038.959    -604.005
data_channel_is_tech           -992.7337    364.826     -2.721      0.007   -1707.802    -277.665
data_channel_is_world          -398.9130    369.500     -1.080      0.280   -1123.142     325.316
num_keywords                      4.5243     36.477      0.124      0.901     -66.972      76.021
kw_min_min                        0.4199      1.594      0.263      0.792      -2.705       3.545
kw_max_min                        0.0448      0.049      0.910      0.363      -0.052       0.141
kw_avg_min                       -0.0612      0.302     -0.202      0.840      -0.654       0.531
kw_min_max                       -0.0014      0.001     -1.207      0.227      -0.004       0.001
kw_max_max                       -0.0002      0.001     -0.283      0.777      -0.001       0.001
kw_avg_max                       -0.0002      0.001     -0.198      0.843      -0.002       0.001
kw_min_avg                       -0.2937      0.074     -3.952      0.000      -0.439      -0.148
kw_max_avg                       -0.1133      0.025     -4.541      0.000      -0.162      -0.064
kw_avg_avg                        0.9789      0.142      6.878      0.000       0.700       1.258
self_reference_min_shares         0.0250      0.007      3.387      0.001       0.011       0.040
self_reference_max_shares         0.0054     

In [50]:
#Backward ELimination
cols = list(x.columns) #creating a list of columns
pmax=1
while(len(cols)>0):
    p=[]                                    #empty list
    X = x[cols]                            #creating a dataframe from the cols
    
    xc = sm.add_constant(X)
    model= sm.OLS(y,xc).fit()
    
    p=pd.Series(model.pvalues.values[1:],index = cols)  #Probability values
    
    
    pmax=max(p)            #Maximum Probability
    
    
    feature_with_pmax = p.idxmax()    #idxmax is going to return the name of index with Max Prob.
    
    
    if(pmax>0.05):
        cols.remove(feature_with_pmax) #Removing that column from cols
    else:
        break

selected_features_BE = cols
print(selected_features_BE)

['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 'num_hrefs', 'data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 'self_reference_min_shares', 'weekday_is_friday', 'is_weekend', 'LDA_03', 'avg_negative_polarity', 'Popularity']


In [57]:
from scipy.stats import shapiro

In [61]:
features=[]
for i in df1.columns:
    pvalue=0
    if (df1[i].dtypes==object)|(df1[i].dtypes==bool):
        cross=pd.crosstab(df1["shares"],df1[i])
        pvalue=chi2_contingency(cross)[1]
        print("CHI",i,pvalue)
        if pvalue<0.05:
            features.append(i)
    else:
        T=df1[i][df1["shares"]==True]
        F=df1[i][df1["shares"]==False]
        shT=shapiro(T)[1]
        shF=shapiro(F)[1]
        if shT<0.05 or shF<0.05:
            pvalue=kruskal(T,F).pvalue
            print("KRU",i,pvalue)
            if pvalue<0.05:
                features.append(i)
        else:
            pvalue=f_oneway(T,F).pvalue
            print("ANOVA",i,pvalue)
            if pvalue<0.05:
                features.append(i)
print(features)

ValueError: Data must be at least length 3.